In [1]:
from openai import AzureOpenAI
import anthropic
import ollama
import requests
import json
from dotenv import load_dotenv
import os
from IPython.display import display, update_display, Markdown
from bs4 import BeautifulSoup
import re

In [4]:
load_dotenv(override=True)

azure_ai_foundary_api_key = os.getenv("AZURE_AI_FOUNDARY_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")

In [6]:
azure_ai_foundary = AzureOpenAI(
    azure_endpoint = azure_oai_endpoint,
    api_key = azure_ai_foundary_api_key,
    api_version = "2025-01-01-preview"
)


## Tools
Tools allows the LLM to execute actions. Eg if there is a calculator tool then the LLM can use that whenever the user asks for a calculation.

In [24]:
product_prices = {'keyboard': 100, 'mouse': 40, 'motherboard': 143}

def get_product_price(product):
    """Returns the price of a computer part"""
    print(f'Price requested for {product}')
    return product_prices.get(product.lower(), 'Unknown')


# define the tool metadata for the LLM
# The structure follows the structure that open ai expects
computer_price_function = {
    "name": "get_product_price",
    "description": "Returns the price of a computer part. Use this whenever you need to know the price of a computer part. Example: If a customer asks for the price of a mouse, use his function.",
    "parameters": {
        "type": "object",
        "properties": {
            "product": {
                "type": "string",
                "description": "The name of the computer part that customer wants to buy"
            }
        },
        "required": ["product"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": computer_price_function}]

# This will be used to handle the tool request from open ai LLM
def tool_handler(message):
    # extract the func related args 
    tool_call = message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)
    computer_part = args.get('product')

    print(f'Tool handler ........... {computer_part}')
    print(message)
    # use the func
    price = get_product_price(computer_part)

    response = {
        "role": "tool",
        "content": json.dumps({"product": computer_part, "price": price}),
        "tool_call_id": tool_call.id
    }

    return response

In [28]:
def chat(msg, history):
    system_message = "You are an expert assistant who specializes in computer related stuff. \
    You help the user precise one line responses. \
    If rely on functions to get the accuracy and in case you dont know about something, clearly mention that."

    model = "gpt-4o-mini"
    
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": msg}]

    # NOTE - pass the tools object
    response = azure_ai_foundary.chat.completions.create(model=model, messages=messages, tools=tools)
    
    # whenever the llm wants to use a tool, it will have that as the finish_reason
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_response = tool_handler(message)

        # messages + [Received LLM tool call msg] + [our tool handler response]
        messages.append(message)
        messages.append(tool_response)

        response = azure_ai_foundary.chat.completions.create(model=model, messages=messages)

    return response.choices[0].message.content

In [20]:
import gradio as gr

In [29]:
gr.ChatInterface(
    fn=chat,
    type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


Tool handler ........... mouse
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ms6Czq2E2epn4Y4mrwX3q4Kf', function=Function(arguments='{"product":"mouse"}', name='get_product_price'), type='function')])
Price requested for mouse
Tool handler ........... inverter
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8ve4aBLzRiR42gSyZe360jQF', function=Function(arguments='{"product":"inverter"}', name='get_product_price'), type='function')])
Price requested for inverter
Tool handler ........... keyboard
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_sfqbAQuYlKpk8dRjHbGCavHJ', function=Function(arguments='{"product":"keyboard"}', name='g